In [0]:
%matplotlib inline

debug = False

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1
UNK_token = 2
PAD_token = 3
# MAX_LENGTH = 1000

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNK"}
        self.n_words = 3  # Count SOS and EOS and UNK

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def Reverse(lst): 
    return [ele for ele in reversed(lst)]

In [0]:
def readLangs(lang1, lang2, reverse=False, model="dev"):
    
    # if model == "dev":
    #     source = "/content/gdrive/My Drive/NLPA/EndSemProject/pruned_DataSet/dev_pruned.en"
    #     target = "/content/gdrive/My Drive/NLPA/EndSemProject/pruned_DataSet/dev_pruned.hi"
    # else 
    if model == "test":
        source = "/content/gdrive/My Drive/NLPA/EndSemProject/pruned_DataSet/test_pruned.en"
        target = "/content/gdrive/My Drive/NLPA/EndSemProject/pruned_DataSet/test_pruned.hi"
    else:
        source = "/content/gdrive/My Drive/NLPA/EndSemProject/pruned_DataSet/train_pruned.en"
        target = "/content/gdrive/My Drive/NLPA/EndSemProject/pruned_DataSet/train_pruned.hi"
    
    eng_lines = open(source, encoding='utf-8').read().strip().split('\n')
    hin_lines = open(target, encoding='utf-8').read().strip().split('\n')

    lines = []
    for i in range(len(eng_lines)):
        lines.append(eng_lines[i] + '\t' + hin_lines[i])

    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    pairs = [[s for s in l.split('\t')] for l in lines]
    #print(pairs)
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 20
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def prepareData(lang1, lang2, reverse=False, model="dev", filter_sentence=False):
    
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse, model)
    #print(input_lang, output_lang, pairs)
    if(filter_sentence):
        pairs = filterPairs(pairs)
    
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs

In [10]:
input_lang, output_lang, pairs = prepareData('eng', 'hi', reverse=False, model="train", filter_sentence=True)
print(random.choice(pairs))
orig_pairs = []
for p in pairs:
    temp = str(p[0]).split(' ')
    temp = Reverse(temp)
    listToStr = ' '.join([str(elem) for elem in temp]) 
#     print("listToStr")
#     print(listToStr)
    listToStr = str(listToStr)
    orig_pairs.append(listToStr+'\t'+str(p[1]))
#     print(orig_pairs[-1])
print("orig_pairs")
print(orig_pairs[0])
print(random.choice(orig_pairs))

Read 244249 sentence pairs
Trimmed to 244249 sentence pairs
Counting words...
Counted words:
eng 78242
hi 88737
['We gave Moses the Book and the criteria so that perhaps you would be rightly guided . ', 'और याद करो जब मूसा को हमने किताब और कसौटी प्रदान की , ताकि तुम मार्ग पा सको']
orig_pairs
mistake careless a make ll ' I sure m ' I	मुझे जनता हूँ की मैं इस वीडियो में किसी भी समय
fears and gives that him For	तो जिस किसी ने दिया और डर रखा , 


The Encoder
-----------

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.):
        super(EncoderRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, bidirectional=True, dropout=dropout) # batch_first=False,
        # self.gru = nn.GRU(self.hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedding = self.embedding(input).view(1, 1, -1)
        output = embedding
        output, hidden = self.gru(output, hidden)
        return output, hidden
        # packed = pack_padded_sequence(x, lengths, batch_first=True)
        # output, final = self.gru(packed)
        # output, _ = pad_packed_sequence(output, batch_first=True)

        # Manually concatenating the final states for both directions
        # fwd_final = final[0:final.size(0):2]
        # bwd_final = final[1:final.size(0):2]
        # final = torch.cat([fwd_final, bwd_final], dim=2)  # [num_layers, batch, 2*dim]

        # return output, final

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

The Attention Decoder
---------------------





In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0., num_layers=1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.num_layers = num_layers
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size*2)

        # self.pre_output_layer = nn.Linear(hidden_size + 2*hidden_size + emb_size, hidden_size, bias=False)

        # Attention -- general/bilinear
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 4, self.hidden_size)
        self.attn_general = nn.Linear(self.max_length, self.max_length)
        self.lcl_wa_into_hs = nn.Linear(self.hidden_size*2, self.hidden_size*2)
        self.lcl_wb_into_hs = nn.Linear(self.hidden_size*2, self.hidden_size)

        # self.gru = nn.GRU(emb_size + 2*hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.num_layers, bidirectional=True)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

        # self.attn_coverage = nn.Linear(self.max_length, self.hidden_size)
        # self.attn_coverage_cat = nn.Linear(self.hidden_size*3, self.max_length)

    def forward(self, input, hidden, encoder1_outputs, encoder2_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        # print("before prev_hidden_tilde")
        # _, prev_hidden_tilde = self.gru(hidden, embedded);
        # print("After prev_hidden_tilde")

        print("encoder1_outputs shape: ",encoder1_outputs.size())
        print("embedded shape: ",self.lcl_wa_into_hs(embedded[0]).size())

        # Attention weigths for Phrase based translations encodings 
        alphas = self.attn_general(torch.matmul(self.lcl_wa_into_hs(embedded[0]), encoder1_outputs.T))
        # alphas = self.attn_general(torch.matmul(self.lcl_wa_into_hs(prev_hidden_tilde), encoder1_outputs.T))
        attn1_weights = F.softmax(alphas, dim = 1)
        attn1_applied = torch.bmm(attn1_weights.unsqueeze(0), encoder1_outputs.unsqueeze(0))
        output1 = torch.cat((embedded[0], attn1_applied[0]), 1)
        # output1 = torch.cat((prev_hidden_tilde, attn1_applied[0]), 1)
        output1 = self.attn_combine(output1).unsqueeze(0)
        # output1 = F.relu(output1)

        # Attention weigths for NMT based translations encodings
        betas = self.attn_general(torch.matmul(self.lcl_wa_into_hs(embedded[0]), encoder2_outputs.T))
        # betas = self.attn_general(torch.matmul(self.lcl_wa_into_hs(prev_hidden_tilde), encoder2_outputs.T))
        attn2_weights = F.softmax(betas, dim = 1)
        attn2_applied = torch.bmm(attn2_weights.unsqueeze(0), encoder2_outputs.unsqueeze(0))
        output2 = torch.cat((embedded[0], attn2_applied[0]), 1)
        # output2 = torch.cat((prev_hidden_tilde, attn2_applied[0]), 1)
        output2 = self.attn_combine(output2).unsqueeze(0)
        # output2 = F.relu(output2)

        # output = torch.cat((output1, output2), 1)

        output1 = output1[:, -1, :]
        output2 = output2[:, -1, :]
        print("output1 shape: ",output1.T.size())
        print("embedded shape: ",self.lcl_wb_into_hs(embedded[0]).size())

        # combinations' attention weights
        # X = torch.matmul(self.lcl_wb_into_hs(embedded[0])
        # m2 = output1.transpose()
        gammas = self.attn_general(torch.matmul(self.lcl_wb_into_hs(embedded[0]), output1.T))
        attn_weights = F.softmax(gammas, dim = 1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), output2.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)

        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

Training
========

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else UNK_token for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

In [0]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, encoder2, decoder, encoder_optimizer, encoder2_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder2_hidden = encoder2.initHidden()

    encoder_optimizer.zero_grad()
    encoder2_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)
    encoder2_outputs = torch.zeros(max_length, encoder2.hidden_size*2, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        encoder2_output, encoder2_hidden = encoder(input_tensor[ei], encoder2_hidden)
        encoder2_outputs[ei] = encoder2_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    # To change - concatenate hidden states of both the encoders and supply that as a
    # decoder_hidden = encoder_hidden
    decoder_hidden = torch.cat((encoder_hidden, encoder2_hidden), 1)
    # decoder_hidden = encoder_hidden.add(encoder2_hidden)

    # use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    use_teacher_forcing = True

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs, encoder2_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs, encoder2_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    encoder2_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
loss_list = []
epoch_list = []

In [0]:
def trainIters(encoder, encoder2, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    encoder2_optimizer = optim.SGD(encoder2.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, encoder2, decoder, encoder_optimizer, encoder2_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            
            loss_list.append(print_loss_avg)
            epoch_list.append(iter)
            print('epoch = ',epoch_list[-1],'  loss = ',loss_list[-1])

            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

Plotting results
----------------

In [0]:
def showPlot(loss_list, epoch_list):
    plt.plot(epoch_list, loss_list)
    plt.xticks(np.arange(0, 75000, 10000))
    plt.yticks(np.arange(0, 5, 0.5))
    plt.savefig("test.png")
    plt.show()
    plt.close('all')

In [0]:
print(output_lang.n_words)
print(input_lang.n_words)
hidden_size = 500
no_of_epoch = 50000
# no_hidden_states = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, encoder2, attn_decoder1, no_of_epoch, print_every=1000)

Loading and Saving the models
--------------------------------

In [0]:

# model_type = "AttnDecoder"

# model_name_dec =  model_type+"_Model_"+str(no_of_epoch)+"_"+str(no_hidden_states)
# torch.save(attn_decoder1.state_dict(), model_name_dec)
# # device = torch.device('cpu')
# decoder_model = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
# decoder_model.load_state_dict(torch.load(model_name_dec, map_location=device))

In [0]:
datafile = "pruned_dataset_"
encoder1_type = "PBMT_"
encoder2_type = "NMT_"
rnn_type = "BiGRU_"
encoder_model_name = encoder1_type+datafile+str(no_of_epoch)+"_"+str(hidden_size)+".encoder"
encoder2_model_name = encoder2_type+datafile+str(no_of_epoch)+"_"+str(hidden_size)+".encoder"
decoder_model_name = rnn_type+datafile+str(no_of_epoch)+"_"+str(hidden_size)+".attndecoder"

In [0]:
# Saving the trained models
device = torch.device('cpu')

torch.save(encoder1.state_dict(), encoder_model_name)
torch.save(encoder2.state_dict(), encoder2_model_name)
torch.save(attn_decoder1.state_dict(), decoder_model_name)

In [0]:
# Loading the saved models
# device = torch.device('cpu')

# encoder_model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# encoder_model.load_state_dict(torch.load(encoder_model_name, map_location=device))

# encoder2_model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# encoder2_model.load_state_dict(torch.load(encoder2_model_name, map_location=device))

# decoder_model = DecoderRNN(hidden_size, output_lang.n_words).to(device)
# decoder_model.load_state_dict(torch.load(decoder_model_name, map_location=device))

In [0]:
showPlot(plot_losses, plot_epoch)

In [0]:
def evaluate(encoder, encoder2, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder2_hidden = encoder2.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)
        encoder2_outputs = torch.zeros(max_length, encoder2.hidden_size*2, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
            encoder2_output, encoder2_hidden = encoder(input_tensor[ei], encoder2_hidden)
            encoder2_outputs[ei] += encoder2_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        # To be changed
        # decoder_hidden = encoder_hidden
        decoder_hidden = torch.cat((encoder_hidden, encoder2_hidden), 1)

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs, encoder2_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            try:
              if topi.item() == UNK_token:
                  decoded_words.append('<UNK>')
              if topi.item() == EOS_token:
                  decoded_words.append('<EOS>')
                  break
              else:
                  decoded_words.append(output_lang.index2word[topi.item()])
            except:
              continue
            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, encoder2, decoder, n=10):
    for i in range(n):
        pair = random.choice(orig_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, encoder2, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [0]:
device = torch.device('cuda')
evaluateRandomly(encoder1, encoder2, attn_decoder1)

In [0]:
print(epoch_list)
print(loss_list)

Evaluation on Test Data
------------------------

In [0]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction    

def calculate_bleu(pred_trg, real_trg):
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(real_trg, pred_trg, smoothing_function=smoothie)
    return score 

def calculate_Result(encoder, encoder2, decoder,lcl_pairs, n=50):
    device = torch.device('cuda')

    result_value_bleu_score = []
    
    for i in range(n):
        pair = random.choice(lcl_pairs)
        if debug:
          print('>', pair[0])
          print('=', pair[1])
        output_words, attentions = evaluate(encoder, encoder2, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        if debug:
          print('<', output_sentence)
        reference = [pair[1].split()]
        if debug:
          print('--', reference)
        output_words = output_words[:-1]
        temp  = []
        for ow in output_words:
          if ow!='':
            temp.append(ow)
        output_words = temp
        target_predicted = output_words
        
        if debug:        
          print('<<', output_words)
        
        score = calculate_bleu(target_predicted,reference)
        
        if debug:
          print("---Value",score)
        
        result_value_bleu_score.append((pair[0],pair[1].split(),target_predicted,score))

    return result_value_bleu_score

In [0]:
input_lang, output_lang, pairs = prepareData('eng', 'hi', reverse=False, model="train", filter_sentence=True)
print(random.choice(pairs))

In [0]:
global debug
debug = 1
device = torch.device('cuda')
result_value_bleu_score = calculate_Result(encoder1, encoder2, attn_decoder1, pairs)
result_value_bleu_score_dict = {}
result_value_bleu_score_dict['result'] = result_value_bleu_score 
# torch.save(result_value_bleu_score_dict, train_result_data_path)
for item in result_value_bleu_score:
  if (item[3]>0):
    print(" Source Language ",item[0])
    print(" Input Target",item[1])
    print(" Output Target",item[2])
    print(" Score ",item[3])

In [0]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder1, encoder2, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

In [0]:
evaluateAndShowAttention("So if you have a strategy , use it against Me .")

evaluateAndShowAttention("The alliance with Goharbai remained controversial to his dying day .")

Visualizing Attention
---------------------

In [0]:
output_words, attentions = evaluate(encoder1, encoder2, attn_decoder1, "He seemed to have ultimately attained something he had been hankering after all these years .")
plt.matshow(attentions.numpy())